In [4]:
import cv2
import os
import numpy as np

In [5]:
DATASET_DIR="../imc_25/image-matching-challenge-2025/train/imc2024_lizard_pond"
THRESHOLD = 15

image_names = sorted(os.listdir(DATASET_DIR))
ref_name = image_names[0]   # choose one reference image

In [6]:
ref_path = os.path.join(DATASET_DIR, ref_name)
ref_img = cv2.imread(ref_path, cv2.IMREAD_GRAYSCALE)

sift = cv2.SIFT_create()
kp_ref, des_ref = sift.detectAndCompute(ref_img, None)

print("Reference image:", ref_name)
print("Keypoints:", len(kp_ref))


Reference image: lizard_00003.png
Keypoints: 6895


In [7]:
bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=True)

edges = []

for other_name in image_names[1:]:
    other_path = os.path.join(DATASET_DIR, other_name)
    other_img = cv2.imread(other_path, cv2.IMREAD_GRAYSCALE)

    kp_o, des_o = sift.detectAndCompute(other_img, None)

    if des_o is None:
        continue

    matches = bf.match(des_ref, des_o)

    if len(matches) < 8:
        continue

    pts1 = np.float32([kp_ref[m.queryIdx].pt for m in matches])
    pts2 = np.float32([kp_o[m.trainIdx].pt for m in matches])

    F, mask = cv2.findFundamentalMat(
        pts1, pts2,
        cv2.FM_RANSAC,
        ransacReprojThreshold=1.0,
        confidence=0.99
    )

    inliers = 0 if mask is None else int(mask.sum())

    if inliers >= THRESHOLD:
        edges.append((ref_name, other_name, inliers))

    print(f"{ref_name} ↔ {other_name}: inliers = {inliers}")


sift = cv2.SIFT_create()
kp_ref, des_ref = sift.detectAndCompute(ref_img, None)

print("Reference image:", ref_name)
print("Keypoints:", len(kp_ref))


lizard_00003.png ↔ lizard_00013.png: inliers = 17
lizard_00003.png ↔ lizard_00014.png: inliers = 16
lizard_00003.png ↔ lizard_00017.png: inliers = 16
lizard_00003.png ↔ lizard_00019.png: inliers = 15
lizard_00003.png ↔ lizard_00034.png: inliers = 18
lizard_00003.png ↔ lizard_00051.png: inliers = 19
lizard_00003.png ↔ lizard_00055.png: inliers = 10
lizard_00003.png ↔ lizard_00066.png: inliers = 11
lizard_00003.png ↔ lizard_00070.png: inliers = 13
lizard_00003.png ↔ lizard_00074.png: inliers = 13
lizard_00003.png ↔ lizard_00075.png: inliers = 12
lizard_00003.png ↔ lizard_00086.png: inliers = 12
lizard_00003.png ↔ lizard_00087.png: inliers = 10
lizard_00003.png ↔ lizard_00089.png: inliers = 11
lizard_00003.png ↔ lizard_00107.png: inliers = 22
lizard_00003.png ↔ lizard_00113.png: inliers = 18
lizard_00003.png ↔ lizard_00140.png: inliers = 16
lizard_00003.png ↔ lizard_00154.png: inliers = 16
lizard_00003.png ↔ lizard_00160.png: inliers = 17
lizard_00003.png ↔ lizard_00161.png: inliers = 18
